# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- SIL - sport intensity level - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- JG - juggling
- GM - gaming


(Separate message about SIL)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [2]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER

DAY_OFFS = []
DESIRED_MEAN_VALUE = 240
report_data, wide_use_data = prepare_data(PATH, DAY_OFFS, DESIRED_MEAN_VALUE)
report_data

c:\Users\San\Documents\CS projects\time_counter\preparation\calc_study_data.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Total"] = total_per_day


[0, 1, 0, 1, 50, 0, 'you studied every day', 1]

In [8]:
dir()

['DAY_OFFS',
 'DESIRED_MEAN_VALUE',
 'In',
 'Out',
 'PATH',
 '_',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'append_report',
 'calc_req_study_time',
 'calc_study_data',
 'categorise',
 'cur_stats',
 'df',
 'exit',
 'generate_report',
 'get_ipython',
 'handle_day_offs',
 'handle_nonstudy_data',
 'mean',
 'min_to_study',
 'month',
 'nonstudy_df',
 'np',
 'os',
 'pd',
 'plot_data',
 'prepare_data',
 'prepare_initial_df',
 'quit',
 'report_data',
 'std',
 'sys',
 'wide_use_data']

In [9]:
# maybe this func is gonna belong to 'monitoring' package
def sport_stats():
    """
    (Example)
    You exercised this month: 10 times
    Last time you got medium exercise: 3 days ago
    Last time you got hard exercise: 7 days ago
    """
    pass